In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import os
import json
import math

In [2]:
CSV_PATH = "/Users/hemengya/Desktop/数字贸易规则新版/第五版/embedding/processed_data.csv"
CACHE_PATH = "/Users/hemengya/Desktop/数字贸易规则新版/第五版/embedding/merged_embedding_results.json"
EXPORT_PATH = "/Users/hemengya/Desktop/数字贸易规则新版/第五版/output/similarity"
TOPIC_DATA_PATH = "/Users/hemengya/Desktop/数字贸易规则新版/第五版/output/grid_research/result_n_comp_20_msize_5_neig_25.xlsx"

# load data

In [3]:
data_embedding = pd.read_csv(CSV_PATH)

# 读取 JSON 文件（包含嵌入向量）
cache = {}
with open(CACHE_PATH, "r", encoding="utf-8") as f:
    for line in f:
        try:
            obj = json.loads(line.strip())
            key = str(obj["key"])
            embedding = obj.get("embedding")
            if embedding:
                cache[key] = embedding
        except json.JSONDecodeError as e:
            print(f"⚠️ 无法解析行，跳过: {line.strip()}，错误: {e}")

print(f"从缓存文件加载了 {len(cache)} 个有效的嵌入向量。")

从缓存文件加载了 3229 个有效的嵌入向量。


In [4]:
try:
    df_topics = pd.read_excel(TOPIC_DATA_PATH)
    if 'key' not in df_topics.columns or 'source' not in df_topics.columns or 'hc_topic_label' not in df_topics.columns:
        raise ValueError("主题文件缺少 'key', 'source', 或 'topic' 列。")
    print(f"从 {TOPIC_DATA_PATH} 加载了主题数据。")
except FileNotFoundError:
    print(f"错误: 主题文件未找到，路径: {TOPIC_DATA_PATH}")
    exit()
except ValueError as ve:
    print(f"错误: {ve}")
    exit()
except Exception as e:
    print(f"加载主题文件时发生意外错误: {e}")
    exit()

从 /Users/hemengya/Desktop/数字贸易规则新版/第五版/output/grid_research/result_n_comp_20_msize_5_neig_25.xlsx 加载了主题数据。


In [5]:

# --- 2. 数据整合与对齐 ---

# 使用 key 作为连接点，整合所有数据
merged_data = []
for index, row in df_topics.iterrows():
    key = str(row['key'])
    embedding = cache.get(key)
    
    # 确保有对应的嵌入向量和有效的 source/topic
    if embedding and pd.notna(row['source']) and pd.notna(row['hc_topic_label']):
        merged_data.append({
            "key": key,
            "source": row['source'],
            "topic": row['hc_topic_label'],
            "embedding": embedding
        })

if not merged_data:
    print("没有找到有效且可匹配的文档数据和嵌入向量。程序终止。")
    exit()

df_complete = pd.DataFrame(merged_data)
df_complete['embedding'] = df_complete['embedding'].apply(lambda x: np.array(x, dtype=float))

print(f"成功整合了 {len(df_complete)} 个文档的数据，包括 key, source, topic 和 embedding。")

成功整合了 3229 个文档的数据，包括 key, source, topic 和 embedding。


In [6]:
unique_topics = df_complete['topic'].unique()
print(f"\n—— 发现 {len(unique_topics)} 个唯一主题，开始计算相似度 ——")

export_dir = os.path.join(os.path.dirname(EXPORT_PATH), "topic_similarity_matrices")
os.makedirs(export_dir, exist_ok=True)

for topic_id in unique_topics:
    # 筛选出当前主题的文档
    df_topic = df_complete[df_complete['topic'] == topic_id].copy()
    
    if len(df_topic) < 2:
        print(f"ℹ️ 主题 '{topic_id}' 的文档少于2个 ({len(df_topic)} 个)。跳过相似度计算。")
        continue

    print(f"\n处理主题 '{topic_id}'，包含 {len(df_topic)} 个文档...")

    # --- 合并同一 source 的向量 ---
    merged_embeddings = df_topic.groupby('source')['embedding'].apply(
        lambda vecs: np.mean(np.stack(vecs.values), axis=0)
    )
    
    sources_in_topic = merged_embeddings.index.tolist()
    embeddings_in_topic = np.stack(merged_embeddings.values)

    # 计算余弦相似度矩阵
    similarity_matrix = cosine_similarity(embeddings_in_topic)
    
    # 创建 DataFrame，并设置 index 和 columns 为 source
    df_sim = pd.DataFrame(similarity_matrix, index=sources_in_topic, columns=sources_in_topic)
    
    # 导出到 Excel
    safe_topic_id = str(topic_id).replace("/", "_").replace("\\", "_").replace(":", "_").replace("*", "_").replace("?", "_")
    output_filename = f"similarity_matrix_topic_{safe_topic_id}.xlsx"
    output_path = os.path.join(export_dir, output_filename)
    
    try:
        df_sim.to_excel(output_path)
        print(f"成功导出主题 '{topic_id}' 的相似度矩阵到: {output_path}")
    except Exception as e:
        print(f"导出主题 '{topic_id}' 的相似度矩阵失败: {e}")

print("\n 所有主题的相似度计算和导出已完成。")


—— 发现 98 个唯一主题，开始计算相似度 ——

处理主题 '17'，包含 46 个文档...
成功导出主题 '17' 的相似度矩阵到: /Users/hemengya/Desktop/数字贸易规则新版/第五版/output/topic_similarity_matrices/similarity_matrix_topic_17.xlsx

处理主题 '68'，包含 104 个文档...
成功导出主题 '68' 的相似度矩阵到: /Users/hemengya/Desktop/数字贸易规则新版/第五版/output/topic_similarity_matrices/similarity_matrix_topic_68.xlsx

处理主题 '7'，包含 10 个文档...
成功导出主题 '7' 的相似度矩阵到: /Users/hemengya/Desktop/数字贸易规则新版/第五版/output/topic_similarity_matrices/similarity_matrix_topic_7.xlsx

处理主题 '31'，包含 21 个文档...
成功导出主题 '31' 的相似度矩阵到: /Users/hemengya/Desktop/数字贸易规则新版/第五版/output/topic_similarity_matrices/similarity_matrix_topic_31.xlsx

处理主题 '44'，包含 65 个文档...
成功导出主题 '44' 的相似度矩阵到: /Users/hemengya/Desktop/数字贸易规则新版/第五版/output/topic_similarity_matrices/similarity_matrix_topic_44.xlsx

处理主题 '76'，包含 35 个文档...
成功导出主题 '76' 的相似度矩阵到: /Users/hemengya/Desktop/数字贸易规则新版/第五版/output/topic_similarity_matrices/similarity_matrix_topic_76.xlsx

处理主题 '87'，包含 302 个文档...
成功导出主题 '87' 的相似度矩阵到: /Users/hemengya/Desktop/数字贸易规则新版/第五版/output/top